# The Heuristic Model

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
def normalizeGame(somegame):
    means =  np.sum(somegame, 0)#/4
#     ranges = np.ptp(somegame)
    normalgame = (somegame - means)#/ranges
    return normalgame

def printGame(somegame):
    print(somegame[0], somegame[1],"\n",somegame[2],somegame[3])

In [7]:
def softmax(x):
    e_x = np.exp(x - np.max(x))
    out = e_x / e_x.sum()
    return out

# Calculate probability of ending up in each cell

In [8]:
def gameProbabilities(somegame, wAa, wAb, wBa, wBb, gAb, gBa, ChoiceA, ChoiceB):
    ''' Calculate probability of ending up in each cell of a game'''  
    game = normalizeGame(somegame)
#     game = somegame
    EVA1 = (game[0][0] * wAa + game[0][1] * gAb * wAb + game[1][0] * wAa + game[1][1] * gAb * wAb)/2.0
    EVA2 = (game[2][0] * wAa + game[2][1] * gAb * wAb + game[3][0] * wAa + game[3][1] * gAb * wAb)/2.0
    PA = softmax(np.array([EVA1, EVA2]))

    EVB1 = (game[0][0] * gBa * wBa + game[0][1]  * wBb + game[2][0] * wBa * gBa + game[2][1] * wBb)/2.0
    EVB2 = (game[1][0] * gBa * wBa + game[1][1]  * wBb + game[3][0] * wBa * gBa + game[3][1] * wBb)/2.0
    PB = softmax(np.array([EVB1, EVB2]))
    
    if False:
        print(EVA1)
        print(EVA2)
        print("PA:",PA)
        print("")
        print(EVB1)
        print(EVB2)
        print("PB:",PB)
        print("")
        print(""+ str(PA[0]*PB[0]) + " " + str(PA[0]*PB[1]) + "\n" + str(PA[1]*PB[0]) + " " + str(PA[1]*PB[1]))
    
    return PA[ChoiceA]* PB[ChoiceB]

# Probability of friends

$ P(friends | C_A, C_B, g) = \frac{P(C_A | friends) * P(friends)}{P(C_A, C_B)} $  
$ \propto P(C_A, C_B \space|\space friends) * P(friends) $  
$ \propto P(C_A, C_B \space|\space friends) $  
$ = P(C_A, C_B \space|\space W_A < W_B, \gamma = 1, ...)$  
$ \propto \frac{1}{n} \sum_{Wa, Wb}^{n} P(C_A, C_B \space|\space W_A < W_B, \gamma = 1, ...)$

In [10]:
def friendProb(game, choiceA, choiceB):
    probs = np.zeros([51, 51])
    wA = np.linspace(0,0.5,51)
    wB = np.linspace(0,0.5,51)
    gamma = 1
    for wAa in wA:
        wAb = 1 - wAa
        for wBb in wB:
            wBa = 1 - wBb
            prob = gameProbabilities(game, wAa, wAb, wBa, wBb, gamma, gamma, choiceA, choiceB)
            probs[int(wAa*100),int(wBb*100)] = prob
#     print(probs)
    return np.sum(probs)/(51.*51.)


In [11]:
def enemyProb(game, choiceA, choiceB):
    probs = np.zeros([51, 51])
    wA = np.linspace(0,0.5,51)
    wB = np.linspace(0,0.5,51)
    gamma = -1
    for wAa in wA:
        wAb = 1 - wAa
        for wBb in wB:
            wBa = 1 - wBb
            prob = gameProbabilities(game, wAa, wAb, wBa, wBb, gamma, gamma, choiceA, choiceB)
            probs[int(wAa*100),int(wBb*100)] = prob
#     print(probs)
    return np.sum(probs)/(51.*51.)


In [12]:
def strangerProb(game, choiceA, choiceB):
    probs = np.zeros([51, 51])
    wA = np.linspace(0.5,1,50)
    wB = np.linspace(0.5,1,50)
    gamma = 1
    for wAa in wA:
        wAb = 1 - wAa
        for wBb in wB:
            wBa = 1 - wBb
            prob = gameProbabilities(game, wAa, wAb, wBa, wBb, gamma, gamma, choiceA, choiceB)
            probs[int(wAa*100)- 50,int(wBb*100) - 50] = prob
#     print(probs)
    return np.sum(probs)/(51.*51.)


In [13]:
def FSE(somegame, choiceA, choiceB):
    sums = [somegame[0][0] + somegame[0][1], somegame[1][0] + somegame[1][1], somegame[2][0] + somegame[2][1], somegame[3][0] + somegame[3][1]] 
    maximum = np.argmax(sums)
    minimum = np.argmin(sums)
    
    F = 0
    S = 0
    E = 0
    index = 2*(choiceA) + choiceB
    if index == maximum:
        F = 1
        
    elif index == minimum:
        E = 1
    else:
        S = 1
    
    #print(index)
    return F,S,E

In [15]:
pd = {"gameString":"pd", "game":[[8, 8],[0, 12],[12, 0],[4,4]], "choices":[0 ,1, 3]}                            # 0 1 3
threat = {"gameString":"threat", "game":[[12, 6],[6, 12],[6, 0],[0,6]], "choices":[ 0, 1, 2, 3]}                      #  0 1 2 3
disjunctive = {"gameString":"disjunctive", "game":[[12,12], [12,12], [12,12], [0,0]], "choices":[0, 3]}      #  0 3
coordination = {"gameString":"coordination", "game":[[12,12], [0,0], [0,0], [12,12]], "choices":[ 0, 1]}       #  0 1
singleControl = {"gameString":"singleControl", "game":[[6,6], [0,6], [6,6], [0,6]], "choices":[0, 1]}              #  0 1
games = [pd, threat, disjunctive, coordination, singleControl]

In [16]:
choicemap = {0:[0,0], 1:[0,1], 2:[1,0], 3:[1,1]}
for game in games:
    for choice in game["choices"]:
        probs = FSE(game["game"],choicemap[choice][0],choicemap[choice][1])
        print("{} \t {} \t FSE: {:f} {:f} {:f}".format(game["gameString"], choicemap[choice], *probs))

pd 	 [0, 0] 	 FSE: 1.000000 0.000000 0.000000
pd 	 [0, 1] 	 FSE: 0.000000 1.000000 0.000000
pd 	 [1, 1] 	 FSE: 0.000000 0.000000 1.000000
threat 	 [0, 0] 	 FSE: 1.000000 0.000000 0.000000
threat 	 [0, 1] 	 FSE: 0.000000 1.000000 0.000000
threat 	 [1, 0] 	 FSE: 0.000000 0.000000 1.000000
threat 	 [1, 1] 	 FSE: 0.000000 1.000000 0.000000
disjunctive 	 [0, 0] 	 FSE: 1.000000 0.000000 0.000000
disjunctive 	 [1, 1] 	 FSE: 0.000000 0.000000 1.000000
coordination 	 [0, 0] 	 FSE: 1.000000 0.000000 0.000000
coordination 	 [0, 1] 	 FSE: 0.000000 0.000000 1.000000
singleControl 	 [0, 0] 	 FSE: 1.000000 0.000000 0.000000
singleControl 	 [0, 1] 	 FSE: 0.000000 0.000000 1.000000
